In [ ]:
import pandas as pd
import os
from utils.data_reading.sound_data.station import StationsCatalog
from scipy import signal
from utils.transformation.signal import butter_bandpass_filter
from tqdm.notebook import tqdm
import datetime
import numpy as np

In [ ]:
### Work in progress ###

for i in tqdm(range(len(df))):
    line = df.iloc[i]

    try:
        ## Station
        # Retrieve station name from the available data in the aplose dataset
        possible_stations = stations.by_date(line["start_datetime"].tz_localize(None))  # [MAHY11, MAHY13, MAHY14]
        station_number = line["filename"].split("_")[1][-1]  # "1"
        station_number_to_station = {s.name[-1]:s for s in possible_stations}  # {'1':MAHY11, '3':MAHY13, '4':MAHY14}
        station = station_number_to_station[station_number]  # MAHY11

        # Compute PSDs on the annotation boxes
        manager = station.get_manager()

        ## Date
        date_start = line['start_datetime'].tz_localize(None)
        date_end = line['end_datetime'].tz_localize(None)
        delta = (date_end - date_start).total_seconds()

        # Widen boxes which are too short
        if delta < nperseg_sec:
            diff = nperseg_sec - delta
            date_end = date_end + datetime.timedelta(seconds=diff)

        ## Frequencies
        # Frequency range when we have weak annotations ('ship_noise')
        if line['type'] == 'WEAK':
            start_frequency = 1e-3 # does not tolerate strict 0 Hz
            end_frequency=120 - 1e-3 # does not tolerate strict 120 Hz
        else:
            start_frequency = line['mean_start_frequency']
            end_frequency = line['mean_end_frequency']

        ## Main PSD (inside annotation box)
        data_signal = manager.get_segment(date_start, date_end)
        f, psd_signal = signal.welch(data_signal, fs=240, nperseg=nperseg)
        psd_signal_filtered = np.where((f >= start_frequency) & (f <= end_frequency), psd_signal, np.nan)


# Save computed psds in a dataframe
df_psd_cleaned = pd.DataFrame(psds_cleaned, index=df.index, columns=f)
df_psd_cleaned.to_pickle(os.path.join(output_path, 'psd_aplose_fq_filtered_no_background.pkl'))